In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_binary
import re

In [ ]:
driver = webdriver.Chrome()

In [ ]:
#主キー：日本語名
#入れ子の辞書：
    #H
    #A
    #B
    #C
    #D
    #S
    #imageLink
    #type
    #rememberedMoves
    #characteristic
pokemon = {}

## GameWithからのスクレイピング

In [ ]:

url = "https://gamewith.jp/pokemon-sv/article/show/375426"

driver.get(url)
html = driver.page_source

In [ ]:
main_soup = BeautifulSoup(html)

In [ ]:
pokemon_list = main_soup.find_all("li", class_="w-pksv-pokemon-list-element")

for element in pokemon_list:
    #各ページにアクセス
    individual_url = element.find("a", class_="_name").get("href")
    driver.get(individual_url)

    html = driver.page_source

    soup = BeautifulSoup(html)

    #日本語名
    title = soup.find("h3").text
    try:
        pattern = '(.*)の色違いと基本情報'
        name = re.match(pattern, title).group(1)
    except:
        pattern = '(.*)の基本情報'
        name = re.match(pattern, title).group(1)
    
    pokemon[name] = {}

    #種族値
    table = soup.find("div", class_="pokesv_bar_table")
    bars = table.find_all("span", class_="bar1")

    pokemon[name]["H"] = bars[0].text
    pokemon[name]["A"] = bars[1].text
    pokemon[name]["B"] = bars[2].text
    pokemon[name]["C"] = bars[3].text
    pokemon[name]["D"] = bars[4].text
    pokemon[name]["S"] = bars[5].text

    #画像リンク
    image_div = soup.find("div", class_="js-pokemon-deff-color-switch")
    image_url = image_div.find("img").get("src")
    pokemon[name]["imageLink"] = image_url

    #タイプ
    type_span = soup.find("span", class_ = "inline")
    imgs = type_span.find_all("img")
    types = set()
    for img in imgs:
        types.add(img.get("alt"))
    pokemon[name]["types"] = list(types)

    #覚える技
    pokemon[name]["rememberedMoves"] = []
    skill_list = soup.find("ol", id="SkillList")
    moves = skill_list.find_all("li", class_="w-pksv-skill-list-element")
    for move in moves:
        move_name = move.find("div", class_="_name").find("a").text
        pokemon[name]["rememberedMoves"].append(move_name)

    #特性
    pokemon[name]["characteristics"] = []

    toku_list = soup.find("ol", class_="pksv_toku_list").find_all("li")

    for toku in toku_list:
        charac_name = toku.find("div", class_="_name").text
        pokemon[name]["characteristics"].append(charac_name)


##　保存

In [ ]:
import json

tf = open("pokemon.json", "w")
json.dump(pokemon, tf, ensure_ascii=False)
tf.close()